# Introduction to the _SlopeOne_ recommendation _algorithm_

This notebook contains a worked example of the [SlopeOne algorithm](https://en.wikipedia.org/wiki/Slope_One).
SlopeOne was developed and described by Daniel Lemire and Anna Maclachlan in 2005 (19 years ago...).

## Slope One as one of "the algorithms"

This algorithm attempts to _predict missing ratings_ and is commonly used in _recommender systems_.

When non-specialists say that they were guided by "the algorithms" - they mean that they were
led to content and views selected by their social media or streaming platform, based on

a) their past choices and
b) the corresponding choices made by other users.

As such, recommendation algorithms like _SlopeOne_ play a very important role in monetising the 
web, making some posts go viral, and (possibly!) increasing the fragmentation of society....

## Objective

To show that _SlopeOne_ is not that complicated - it uses normal arithmetic operations (adding, subtracting,
multiplying and dividing) - no complicated stuff like square roots or cosines, etc.

## This notebook

This notebook has descriptions and python code explaining how to

1. Read an example ratings matrix
2. Create a table of (averaged) ratings differences
3. For each missing user-item rating, attempt to infer it from ratings already provided by the user, as well as the ratings difference table

## Step 1

Import libraries of python software.

These libraries provide useful function that we can call upon, without having to write those functions ourselves.

The libraries are:

- [pandas](https://pandas.pydata.org/) is used to perform data analysis on dataframes, etc.
- [numpy](https://numpy.org/) is used to perform operations on arrays of numbers
  

In [ ]:
import pandas as pd
import numpy as np
from IPython.display import display, HTML

## Step 2

Read the ratings data (stored in 'data/example.csv') into a dataframe.

We use the `read_csv()` function provided by pandas so we can skip the messy details of how to read data into our program.

Note that we use a comma (`,`) as a separator and we supply the names of the artists here too.

In [ ]:
ratings = pd.read_csv('data/example.csv', sep=',', names=['TS','LDR','TW','D'])
print('The ratings, provided as input. Note that Alice has not rated `TW` or `D`. Here they are:')
display(ratings)

## Step 2 extra

Note that there are 3 users rating 4 artists:

In [ ]:
[nUsers, nItems] = ratings.shape
print('There are {} users and {} artists'.format(nUsers, nItems))

## Step 3 - introduction

The ratings data contains star ratings on a 1 to 5 scale, where 1 star is a low mark, indicating the rater does not like the artist, and 5 stars indicate the rater is a huge fan of that artist.

The 3 example users are named Alice, Bob and Carol (note the first letters of each name...).

The 4 artists are Taylor Swift (TS), Lana Del Rey (LDR), The Weeknd (TW) and Drake (D).

Note that Alice has not rated 'TW' (The Weeknd) or 'D' (Drake).

However, she has rated 'TS' (Taylor Swift) and 'LDR' (Lana del Rey).

Bob and Carol have rated all 4 artists.

We wish to predict how Alice might rate 'TW' and 'D'.
We would then recommend to Alice the artist she is predicted to rate higher than the other.

We use the _Slope-One_ algorithm to predict the two missing ratings.


## Stage 1: Preprocessing

We need to compute the average difference between the ratings for each pair of artists.

We first create a matrix with the right size.

So we compute an `nItems` x `nItems` matrix of rating differences 

In [ ]:
rd = np.zeros([nItems,nItems], dtype=float)

## Step 3

Now loop over the indices to compute the differences

Note that

1. We loop over each item `i` (an artist in this case) and then over each item `j` for that item `i` (so that we look at _each pair of_ items).
 We say that the `j`-loop is _nested_ inside the `i`-loop.
2. `ratings.iloc[:,k]` is panda's way of getting at the `k`th _column_ of the ratings matrix.
 - Q: How can we access the `k`th _row_ of the ratings matrix?
3. `rI-rJ` is the difference between the ratings, where k>1 users have rated both item `i` and item `j`.
 - If there are k rating differences, we need to take the mean of those differences to find a single representative value for the rating difference between items (artists) `i` and `j`. 
4. After computing this rating difference, we store it in an `rd` matrix for later use.
 

In [ ]:
for i in range(nItems):
  for j in range(nItems):
    rI = ratings.iloc[:,i]
    rJ = ratings.iloc[:,j]
    rd[i,j] = (rI-rJ).mean()

## Step 3 - extra

Note that the rating difference matrix `rd` is anti-symmetric:

- values along the main diagonal are zero - each item has the same rating as itself
- values across the diagonal have opposite signs:
  the rating difference between item `i` and item `j` has the opposite sign of the rating
  difference between item `j` and item `i`

In [ ]:
print('The matrix containing the difference in ratings between each pair of items is:')
display(rd)

## Step 4 - introduction

Stage 2: predicting the missing ratings

Remember: Alice has not rated either _The Weeknd_ (`TW`) or _Drake_ (`D`).
Based on the ratings she has provided, and how others have rated these and other artists, we wish to predict how she might rate _The Weeknd_ and _Drake_.

First, we make a copy of the ratings matrix, so we can compare the ratings before and after predicting the missing ones.

In [ ]:
completedRatings = ratings.copy()

## Step 4

Now we start some more nested loops:

1. First we look at each row (remember: each row contains the ratings given by a single user), so we look at each `userK` in turn.
2. Next we look across that row for each missing rating.
   - The rating by `userK` for `itemI` is missing if it is `NaN` (Not a Number). `np.isnan(x)` returns True if `x` is not a number.
   - Note that we initialise `sum` and `count`, because we intend to accumulate data that we can use to predict this missing rating.
3. Now we search for any ratings that _were_ provided in that row.
   - Note that we flip the condition - this time we are looking for (`userK`,`itemJ`) ratings that _are_ numbers.
   - We take the rating from `userK` for `itemJ` and add the ratings difference `rd[itemI,itemJ]` that we computed previously (during the preprocessing stage).
   - This is a candidate rating prediction, but there may be others.
   - As with the preprocessing stage, we will take the average of these estimates and use that as our best estimate of the missing rating.
   - The only difference is that here we calculate the average ourselves....
   - Remember: the average of a list of numbers is the sum of that list, divided by the count of numbers in that list.
4. Note that we check that `count>0` before using it to calculate the predicted rating. Why do we do this?
   - If count=0, what does this mean?

In [ ]:
for userK in range(nUsers):
  for itemI in range(nItems):
    if (np.isnan(completedRatings.iloc[userK,itemI])):
      sum = 0
      count = 0
      for itemJ in range(nItems):
        if (not np.isnan(completedRatings.iloc[userK,itemJ])): 
          sum += completedRatings.iloc[userK,itemJ] + rd[itemI,itemJ]
          count += 1
      if count>0:
        completedRatings.iloc[userK,itemI] = sum/count

## Step 4 - extra

We can print the completedRatings matrix. You should be able to see that

1. existing ratings are still the same
2. missing ratings now have predicted values

In [ ]:
print('The completed ratings matrix is:')
display(completedRatings)

## Some questions for you

Q1: How might __the algorithm__ choose which Artist to recommend to Alice?

Q2: In practice, there could be millions of users and thousands of artists. How would such large amounts of data

- benefit, and/or
- cause problems for

SlopeOne?

Q3: How could SlopeOne be used to generate recommendations even where users do not rate items?